In [1]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import *
import logging
logging.basicConfig(level=logging.ERROR)

In [2]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
train = pd.read_csv('../input/smai-project-data/train_data.csv')
val = pd.read_csv('../input/smai-project-data/val_data.csv')


In [4]:
train['merged'] = train['question1'] + " " + train['question2']
val['merged'] = val['question1'] + " " + val['question2']


In [5]:
train.shape, val.shape

((283003, 9), (80858, 9))

In [6]:
train.head()

,id,qid1,qid2,question1,question2,is_duplicate,question1_preprocessed,question2_preprocessed,merged
0,8067,15738,15739,How do I play Pokémon GO in Korea?,How do I play Pokémon GO in China?,0,how do i play pok mon go in korea ?,how do i play pok mon go in china ?,How do I play Pokémon GO in Korea? How do I pl...
1,368101,12736,104117,What are some of the best side dishes for crab...,What are some good side dishes for buffalo chi...,0,what are some of the best side dishes for crab...,what are some good side dishes for buffalo chi...,What are some of the best side dishes for crab...
2,70497,121486,121487,Which is more advisable and better material fo...,What is the best server setup for buddypress?,0,which is more advisable and better material fo...,what is the best server setup for buddypress ?,Which is more advisable and better material fo...
3,226567,254474,258192,How do I improve logical programming skills?,How can I improve my logical skills for progra...,1,how do i improve logical programming skills ?,how can i improve my logical skills for progra...,How do I improve logical programming skills? H...
4,73186,48103,3062,How close we are to see 3rd world war?,How close is a World War III?,1,how close we are to see 3rd world war ?,how close is a world war iii ?,How close we are to see 3rd world war? How clo...


In [7]:
val.head()

,id,qid1,qid2,question1,question2,is_duplicate,question1_preprocessed,question2_preprocessed,merged
0,81995,139138,139139,When were emojis created? How did they become ...,Who created the emojis and when where they cre...,1,when were emojis created ? how did they become...,who created the emojis and when where they cre...,When were emojis created? How did they become ...
1,289624,233615,68923,How much time did you take to learn guitar?,How much time it will take for the beginners t...,1,how much time did you take to learn guitar ?,how much time it will take for the beginners t...,How much time did you take to learn guitar? Ho...
2,66887,62546,115846,How long does it take to learn how to develop ...,How much time does it take to learn and do And...,0,how long does it take to learn how to develop ...,how much time does it take to learn and do and...,How long does it take to learn how to develop ...
3,236648,347622,347623,What is the kinetic energy of a 150kg object t...,Is kinetic energy gained when it is moving at ...,0,what is the kinetic energy of a 150kg object t...,is kinetic energy gained when it is moving at ...,What is the kinetic energy of a 150kg object t...
4,289076,410137,410138,What causes this reproducible optical illusion?,What causes an optical illusion?,0,what causes this reproducible optical illusion ?,what causes an optical illusion ?,What causes this reproducible optical illusion...


In [8]:
train['is_duplicate'].unique(), val['is_duplicate'].unique()

(array([0, 1]), array([1, 0]))

In [9]:
train.describe()

,id,qid1,qid2,is_duplicate
count,283003.000000,283003.000000,283003.000000,283003.000000
mean,202192.331979,217054.933467,220872.439928,0.369667
std,116731.031422,157731.997689,160025.552542,0.482715
min,0.000000,1.000000,2.000000,0.000000
25%,101051.500000,74268.000000,74497.000000,0.000000
50%,202121.000000,191808.000000,196616.000000,0.000000
75%,303346.500000,346170.000000,354658.000000,1.000000
max,404289.000000,537932.000000,537933.000000,1.000000


In [10]:
val.describe()

,id,qid1,qid2,is_duplicate
count,80858.000000,80858.000000,80858.000000,80858.000000
mean,202180.180131,218050.184744,221341.987831,0.366272
std,116628.431722,157931.573085,159683.076997,0.481788
min,3.000000,4.000000,8.000000,0.000000
25%,101138.000000,75069.750000,75543.250000,0.000000
50%,202441.000000,193820.500000,198467.500000,0.000000
75%,303186.750000,348423.500000,355237.750000,1.000000
max,404283.000000,537924.000000,537925.000000,1.000000


In [11]:
new_df = train[['merged', 'is_duplicate']]

In [12]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', truncation=True, do_lower_case=True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [13]:
class CitationData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = dataframe.merged
        self.targets = dataframe.is_duplicate
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [14]:
# train_data, test_data = train_test_split(new_df, test_size=0.15)
# train_data = train_data.reset_index(drop=True)
# test_data = test_data.reset_index(drop=True)


# print("FULL Dataset: {}".format(new_df.shape))
print("Train Dataset: {}".format(train.shape))
print("Validation Dataset: {}".format(val.shape))

training_set = CitationData(train, tokenizer, MAX_LEN)
testing_set = CitationData(val, tokenizer, MAX_LEN)

Train Dataset: (283003, 9)
Validation Dataset: (80858, 9)


In [15]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [16]:
class BertClass(torch.nn.Module):
    def __init__(self):
        super(BertClass, self).__init__()
        self.l1 = BertModel.from_pretrained("bert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 2)
        self.relu = torch.nn.ReLU()

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = self.relu(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [17]:
model = BertClass()
model.to(device)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [18]:
model = torch.load('../input/smai-transformers-model/pytorch_bert.bin')

In [19]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [20]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [21]:
def train(epoch, training_loader):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%500==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 500 steps: {loss_step}")
            print(f"Training Accuracy per 500 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [22]:
EPOCHS = 1
for epoch in range(EPOCHS):
    train(epoch, training_loader)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Training Loss per 500 steps: 0.10669399797916412
Training Accuracy per 500 steps: 100.0


500it [03:48,  2.21it/s]

Training Loss per 500 steps: 0.2138806384567082
Training Accuracy per 500 steps: 90.7435129740519


1000it [07:36,  2.20it/s]

...

Training Loss per 500 steps: 0.2096695534582203
Training Accuracy per 500 steps: 91.30278832497609


35000it [4:27:31,  2.18it/s]

Training Loss per 500 steps: 0.20973490815087512
Training Accuracy per 500 steps: 91.30274849290021


35376it [4:30:24,  2.18it/s]

The Total Accuracy for Epoch 0: 91.30044557831542
Training Loss Epoch: 0.20980390078058372
Training Accuracy Epoch: 91.30044557831542


In [23]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids)
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu


In [24]:
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

2it [00:00, 11.32it/s]

Validation Loss per 100 steps: 0.03703940659761429
Validation Accuracy per 100 steps: 100.0


5002it [06:48, 12.09it/s]

Validation Loss per 100 steps: 0.2432929694773432
Validation Accuracy per 100 steps: 89.65706858628275


10002it [13:37, 11.80it/s]

...

Validation Loss per 100 steps: 0.241392637118825
Validation Accuracy per 100 steps: 89.75901606559563


20003it [27:14, 12.31it/s]

Validation Loss per 100 steps: 0.23962147290163993
Validation Accuracy per 100 steps: 89.82300884955752


20215it [27:31, 12.24it/s]

Validation Loss Epoch: 0.23983465061651796
Validation Accuracy Epoch: 89.80929530782359
Accuracy on test data = 89.81%


In [25]:
class CitationData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.merged
        self.targets = self.data.is_duplicate
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [26]:
test_data_f = pd.read_csv('../input/smai-project-data/test_data.csv')
test_data_f['merged'] = test_data_f['question1'] + " " + test_data_f['question2']

In [27]:
test_data_f.head(2)

,id,qid1,qid2,question1,question2,is_duplicate,question1_preprocessed,question2_preprocessed,merged
0,204673,93885,307635,"If there is a God, where is He!","Why is god a ""He""?",0,"if there is a god , where is he !",why is god a `` he '' ?,"If there is a God, where is He! Why is god a ""..."
1,17716,2093,15628,Do you believe that everything happens for a r...,Does everything happen for a reason?,1,do you believe that everything happens for a r...,does everything happen for a reason ?,Do you believe that everything happens for a r...


In [28]:
data_to_test = CitationData(test_data_f[['merged', 'is_duplicate']], tokenizer, MAX_LEN)

test_params = {'batch_size': 8,
                'shuffle': False,
                'num_workers': 0
                }

testing_loader_f = DataLoader(data_to_test, **test_params)

In [29]:
def test(model, testing_loader):
    res = []
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
#             targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids)
            big_val, big_idx = torch.max(outputs, dim=1)
            res.extend(big_idx.tolist())
    
    return res

In [30]:
res = test(model, testing_loader_f)

0it [00:00, ?it/s]/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
5054it [13:22,  6.30it/s]


In [31]:
acc = valid(model, testing_loader_f)
print("Accuracy on test data = %0.2f%%" % acc)

2it [00:00,  5.91it/s]

Validation Loss per 100 steps: 0.055543020367622375
Validation Accuracy per 100 steps: 100.0


5002it [13:15,  6.33it/s]

Validation Loss per 100 steps: 0.2407352560750285
Validation Accuracy per 100 steps: 89.67956408718257


5054it [13:23,  6.29it/s]

Validation Loss Epoch: 0.24062864086756555
Validation Accuracy Epoch: 89.68562170719038
Accuracy on test data = 89.69%


In [32]:
# submission = pd.DataFrame()
# submission['unique_id'] = test_data_f['unique_id']
# submission['citation_influence_label'] = res

In [33]:
# submission.to_csv('submission.csv', index=False)

In [34]:
# train_data.to_csv('train.csv', index=False)
# test_data.to_csv('val.csv', index=False)

In [35]:
output_model_file = 'pytorch_bert.bin'
output_vocab_file = './'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')

All files saved
